In [98]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [95]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

In [3]:
# soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

url = f'https://www.yesstyle.com/es/mujer-limpiadores-faciales/list.html/bcc.14335_bpt.46#/bpt=46&sb=136&pn=1&l=17&s=10&bcc=14335&bt=37' 
r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml')


cream_list = soup.find_all('div', class_='itemContainer')

url = []
for item in cream_list:
        for link in item.find_all('a', href=True): 
            url.append(link['href'])

In [96]:
cleansers_url = []
for i in range (1, 49):
    url = f'https://www.yesstyle.com/en/women-facial-cleansers/list.html/bcc.14335_bpt.46#/bpt=46&sb=136&bt=37&bcc=14335&s=10&pn={i}&l=1' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='itemContainer')
    for item in cleansers_list:
        for link in item.find_all('a', href=True): 
            cleansers_url.append(link['href'])


In [97]:
print(len(cleansers_url))

1728


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

test_url = 'https://www.yesstyle.com/es/cosrx-gel-limpiador-con-ph-bajo-good-morning-150ml/info.html/pid.1052684732'
r = requests.get(test_url) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
name = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
for x in name:
    title = soup.find('h1').text
ingred = soup.find('div', class_="cropWrapper").text #ingredients
rating = soup.find('div', class_='satisfied').text

In [87]:
cleansers_url[1728]

IndexError: list index out of range

In [63]:
title

'\nCOSRX - Gel limpiador con pH bajo Good Morning\n'

'\nWater, Cocamidopropyl Betaine, Sodium Lauroyl Methyl Isethionate, Polysorbate 20, Styrax Japonicus Branch/Fruit/Leaf Extract, Butylene Glycol, Saccharomyces Ferment, Cryptomeria Japonica Leaf Extract, Nelumbo Nucifera Leaf Extract, Pinus Palustris Leaf Extract, Ulmus Davidiana Root Extract, Oenothera Biennis (Evening Primrose) Flower Extract, Pueraria Lobata Root Extract, Melaleuca Alternifolia (Tea Tree) Leaf Oil, Allantoin, Caprylyl Glycol, Ethylhexylglycerin, Betaine Salicylate, Citric Acid, Ethyl Hexanediol, 1,2-Hexanediol, Trisodium Ethylenediamine Disuccinate, Sodium Benzoate, Disodium EDTA\n\nMás\n\n'

'97% Clientes satisfechos|10,635 reseñas'

In [99]:
cleansers_url[0]

'https://www.yesstyle.com/en/cosrx-low-ph-good-morning-gel-cleanser-150ml/info.html/pid.1052684732'

In [100]:
cleansers = []

In [105]:
for link in cleansers_url[1383:1728]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        name_box = soup.find_all('div', class_='headingContent flex-gt-sm-66 flex-100')
        for x in name_box:
            name = soup.find('h1').text
        ingredients = soup.find('div', class_="cropWrapper").text #ingredients
        rating = soup.find('div', class_='satisfied').text
    except:
        name= 'no info'
        ingredients = 'no info'
        rating = 'no info'
        
    cleansers_info = {
    'name': name,
    'rating': rating,
    'ingredients': ingredients}
    
    cleansers.append(cleansers_info)

In [106]:
print(len(cleansers))

1728


In [81]:
cleansers[100]

{'name': '\nKUMANO COSME - Pharmaact Deep Cleansing Oil\n',
 'rating': '97.3% Clientes satisfechos|317 reseñas',
 'ingredients': '\nMineral oil, PEG-8 glyceryl isostearate, water, glycerin, dimethicone, citric acid, jojoba oil, squalane, ethyl paraben, perfume\n\nMás\n\n'}

## 3. PASARLO A DATAFRAME

In [107]:
cleansers_df = pd.DataFrame(cleansers)
cleansers_df

,name,rating,ingredients
0,\nCOSRX - Low pH Good Morning Gel Cleanser\n,"97% customers satisfied|10,636 reviews","\nWater, Cocamidopropyl Betaine, Sodium Lauroy..."
1,\nBANILA CO - Clean It Zero Cleansing Balm Ori...,"99.1% customers satisfied|3,464 reviews","\nCetyl Ethylhexanoate, PEG-20 Glyceryl Triiso..."
2,\nSOME BY MI - Bye Bye Blackhead 30 Days Mirac...,94.8% customers satisfied|795 reviews,"\nWater, Camellia Sinensis (Green Tea) Leaf Wa..."
3,\niUNIK - Calendula Complete Cleansing Oil 200...,98.1% customers satisfied|936 reviews,"\nHelianthus Annuus (Sunflower) Seed Oil, Cano..."
4,\nheimish - All Clean Balm 120ml\n,"98.4% customers satisfied|4,113 reviews","\nEthylhexyl Palmitate, Cetyl Ethylhexanoate, ..."
...,...,...,...
1723,\nheimish - All Clean Green Foam 150ml\n,95.6% customers satisfied|269 reviews,"\nWater, Potassium Laureth Phosphate, Glycerin..."
1724,\nBenton - Honest Cleansing Foam 150g\n,92.9% customers satisfied|333 reviews,"\nAqua (Water), Myristic Acid, Glycerin, Lauri..."
1725,\nEtude House - AC Clean Up Pink Powder Spot 1...,93% customers satisfied|534 reviews,"\nEthanol, Chamaecyparis Obtusa Water, CI 7794..."
1726,\nBANILA CO - Clean It Zero Special Trial Kit\n,100% customers satisfied|153 reviews,\n[Clean It Zero Cleansing Balm Original]: Eth...


In [108]:
cleansers = cleansers_df.to_csv('yesstyle_data/cleansers.csv')